In [1]:
!pip install transforms3d --quiet
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import transforms3d.quaternions as quat
from scipy import signal
import scipy.fft as sfft

In [2]:
focal_length_x = 5212.5371
focal_length_y = 6255.0444
principal_point_x = 640.
principal_point_y = 512.
K = np.array([[focal_length_x, 0, principal_point_x],
                          [0, focal_length_y,principal_point_y],
                          [0, 0, 1]])
P = np.array([[focal_length_x, 0, principal_point_x, 0],
                          [0, focal_length_y,principal_point_y, 0],
                          [0, 0, 1, 0]])

laser_range_distance = 300

labels = np.array([[7.64022827,-46.98920059,-42.21747971,0.98997474,-0.0941886,-0.07856753,0.07003857],
    [12.37142944,-36.08272934,-76.19368744,0.98547542,-0.09323603,-0.13402745,0.04670797],
    [48.83045959,-62.45590591,-149.01345825,0.9463315,-0.15760925,-0.27446723,0.06544811],
    [42.9213562,3.1468811,-156.43341064,0.96294099,-0.05801675,-0.26296937,-0.0150205],
    [25.3291626,-15.59156799,-118.26628113,0.97709697,-0.05717814,-0.20439324,0.01534067],
    [55.51976013,-30.00940132,-172.91856384,0.94631386,-0.10770535,-0.30420068,0.01874543],
    [ 69.65390015,-14.23764992,-194.42814636,0.93474096,-0.0921587,-0.34309945,-0.00697307],
    [67.03590393,57.95062637,-181.86766052,0.94076264,0.03300816,-0.32461578,-0.09219801],
    [95.08578491,70.74472046,-209.08718872,0.91551983,0.02235752,-0.38302255,-0.12090118],
    [115.60142517,80.90587616,-223.3885498,0.89717531,0.01968047,-0.41800421,-0.14128494]])

In [3]:
root = '/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/'

In [4]:
orb = cv2.ORB_create(10000)
FLANN_INDEX_LSH = 6
index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(indexParams=index_params, searchParams=search_params)

In [5]:
img1path = os.path.join(root, 'data/images/01f0f7459b/000.png')
img2path = os.path.join(root, 'data/images/01f0f7459b/001.png')

img1 = cv2.imread(img1path, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(img2path, cv2.IMREAD_GRAYSCALE)

In [6]:
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

In [7]:
matches = flann.knnMatch(descriptors1, descriptors2, k=2)
good = []
for match_pair in matches:
    if len(match_pair) < 2:
        continue  # Skip this match pair if it doesn't have two elements
    m, n = match_pair
    if m.distance < 0.8 * n.distance:
        good.append(m)

len(good)

48

In [8]:
good[0]

< cv2.DMatch 0x7fb75f7f3470>

In [9]:
q1 = np.float32([keypoints1[m.queryIdx].pt for m in good])
q2 = np.float32([keypoints2[m.queryIdx].pt for m in good])

In [10]:
green = (0, 255, 0)
draw_params = dict(
    matchColor=green, # draw matches in green
    singlePointColor=None,
    matchesMask=None, # only draw inliers
    flags=2
)

In [11]:
# img3 = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good, None, **draw_params)
# cv2.imshow("image", img3)
# cv2.waitKey(0)
# cv.destroyAllWindows()

In [12]:
Essential, mask = cv2.findEssentialMat(q1, q2, K)

In [13]:
def form_transf(R, t):
        """
        Makes a transformation matrix from the given rotation matrix and translation vector

        Parameters
        ----------
        R (ndarray): The rotation matrix
        t (list): The translation vector

        Returns
        -------
        T (ndarray): The transformation matrix
        """
        T = np.eye(4, dtype=np.float64)
        T[:3, :3] = R
        T[:3, 3] = t
        return T

def decomp_essential_mat(E, q1, q2):
        """
        Decompose the Essential matrix

        Parameters
        ----------
        E (ndarray): Essential matrix
        q1 (ndarray): The good keypoints matches position in i-1'th image
        q2 (ndarray): The good keypoints matches position in i'th image

        Returns
        -------
        right_pair (list): Contains the rotation matrix and translation vector
        """
        def sum_z_cal_relative_scale(R, t):
            # Get the transformation matrix
            T = form_transf(R, t)
            # Make the projection matrix
            P_cur = np.matmul(np.concatenate((K, np.zeros((3, 1))), axis=1), T)

            # Triangulate the 3D points
            hom_Q1 = cv2.triangulatePoints(P, P_cur, q1.T, q2.T)
            # Also seen from cam 2
            hom_Q2 = np.matmul(T, hom_Q1)

            # Un-homogenize
            uhom_Q1 = hom_Q1[:3, :] / hom_Q1[3, :]
            uhom_Q2 = hom_Q2[:3, :] / hom_Q2[3, :]

            # Find the number of points there has positive z coordinate in both cameras
            sum_of_pos_z_Q1 = sum(uhom_Q1[2, :] > 0)
            sum_of_pos_z_Q2 = sum(uhom_Q2[2, :] > 0)

            # Form point pairs and calculate the relative scale
            relative_scale = np.mean(np.linalg.norm(uhom_Q1.T[:-1] - uhom_Q1.T[1:], axis=-1)/
                                     np.linalg.norm(uhom_Q2.T[:-1] - uhom_Q2.T[1:], axis=-1))
            return sum_of_pos_z_Q1 + sum_of_pos_z_Q2, relative_scale

        # Decompose the essential matrix
        R1, R2, t = cv2.decomposeEssentialMat(E)
        t = np.squeeze(t)

        # Make a list of the different possible pairs
        pairs = [[R1, t], [R1, -t], [R2, t], [R2, -t]]

        # Check which solution there is the right one
        z_sums = []
        relative_scales = []
        for R, t in pairs:
            z_sum, scale = sum_z_cal_relative_scale(R, t)
            z_sums.append(z_sum)
            relative_scales.append(scale)

        # Select the pair there has the most points with positive z coordinate
        right_pair_idx = np.argmax(z_sums)
        right_pair = pairs[right_pair_idx]
        relative_scale = relative_scales[right_pair_idx]
        R1, t = right_pair
        print(relative_scale)
        # t = t * relative_scale
        t

        return [R1, t]

In [19]:
R1, R2, t = cv2.decomposeEssentialMat(Essential)

In [14]:
rotation, transformation = decomp_essential_mat(Essential, q1, q2)

0.9999998344473872


In [15]:
original_trans = labels[0][:3]
original_trans

array([  7.64022827, -46.98920059, -42.21747971])

In [16]:
transformation

array([-0.00774799,  0.0090509 ,  0.99992902])

In [24]:
mean_absolute_error(original_trans, t.ravel())

31.94339355073174

In [18]:
# T = np.eye(4, dtype=np.float64)
# T[:3, :3] = rotation
# T[:3, 3] = transformation
# T

array([[ 0.17873354,  0.98292291,  0.04378225, -0.00774799],
       [ 0.98290868, -0.1763825 , -0.0527232 ,  0.0090509 ],
       [-0.04410042,  0.05245736, -0.99764893,  0.99992902],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])